# Model runs Notebook


In this notebook we train various models and store their predictions. In order to obtain robust results, each version of the dataset (original samples only and then its copy combined with augmentation techniques) is split into 10 folds, allowing 10 unique combinations of train/test data to be used.


In total, we have 8 versions of the dataset, with each being split into 10, we end up with at least 80 train+test runs, which creates a lot of computation demand. These tests were run in Google Colab, results were then stored in pickled dataframe. Prediction results were then analyzed in [Evaluation Notebook](Evaluation.ipynb).


In [ ]:
!pip install -U scikit-learn pandas
# Original sklearn on Colab is 1.2.2 while 1.3.0 is used on my local machine

### Step 1: Preparing results dataframe


In [ ]:
from google.colab import drive
import pandas as pd
path = '/content/gdrive/MyDrive/Colab Notebooks/data/'
drive.mount('/content/gdrive')

results = pd.read_pickle(
    path + 'spectrum.pkl').reset_index()[['slice_file_name', 'fold', 'class']]
augment_results = results.copy()
results['source'] = 'o'
augment_results['source'] = 'a'
results = pd.concat([results, augment_results], ignore_index=True, axis=0)
results.to_pickle(path + "results.pkl")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Main Execution Part


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from google.colab import drive
path = '/content/gdrive/MyDrive/Colab Notebooks/data/'
drive.mount('/content/gdrive')
names = ['dist', 'mixup', 'imixup', 'room', 'warp', 'delay', 'spectrum']
classes = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark',
           'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']
num_classes = len(classes)


def build_df(augment_name):
    """
    This function is effectively building a working dataframe from unagumented data and selected (augment_name) augmentted dataframe
    One-hot encoding is also dne here
    """
    original_df = pd.read_pickle(path + 'spectrum.pkl')
    aug_df = pd.read_pickle(path + augment_name + '.pkl')
    enc = OneHotEncoder(handle_unknown='ignore',
                        feature_name_combiner=lambda x, y: y)
    categories_df = pd.DataFrame(enc.fit_transform(original_df['class'].to_numpy(
    ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())
    categories_df = pd.concat(
        [original_df[['slice_file_name', 'fold', 'r']], categories_df], axis=1)
    categories_df['source'] = 'o'
    if augment_name != 'spectrum':
        if augment_name == 'mixup':
            encoding = aug_df.set_index('classID').to_dict()['class']
            aug_df['class2'] = aug_df['classID2'].map(encoding)
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.8
            categories2_df_add = pd.DataFrame(enc.transform(aug_df['class2'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.2
            categories2_df = categories2_df.add(categories2_df_add)
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        elif augment_name == 'imixup':
            encoding = aug_df.set_index('classID').to_dict()['class']
            aug_df['class2'] = aug_df['classID2'].map(encoding)
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.66
            categories2_df_add = pd.DataFrame(enc.transform(aug_df['class2'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.34
            categories2_df = categories2_df.add(categories2_df_add)
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        else:
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        categories2_df['source'] = 'a'
        return [categories_df, categories2_df]
    else:
        return [categories_df]


def split(df, fold):
    """
    This function is splitting previously built dataframe into train/test/val x/y arrays. Based on folds predefined in dataset
    """
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    # print(len(val))
    return train_x, train_y, test_x, test_y, val_x, val_y

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
import time
import os

# All models will be given a chance to train for 100 epochs. However, in order to prevent overfit,
# Models will be stopped if their validation metrics is not imporving for at least 1% for over 3 epochs


def file_control(fpath):
    """
    This function is used to clear the file storing weights of the model. If not done,
    ModelCheckpoint will capture best performance from the file, ignoring consequent experiment runs.
    """
    if os.path.isfile(fpath):
        # overwrite and make the file blank instead
        open(fpath, 'w').close()
        os.remove(fpath)
        print('cleared old weights')
    else:
        print('already clear')


def build_model(nc=1, shape=(224, 224, 3), base_trainable=False, backbone = "EfNetV2B1", weights = None):
    """
    This is a function creating a EfficientNet based model. The base model itself is pulled from tfhub/tf repo, Following paraneters are to be specified when calling this function:
    nc -- number of classes, or "width" of the model head
    shape -- image size provided for an input
    base_trainable -- layers of the base model (EfficientNetV2B2 in this case) can be locked as trainable or not, affecting model performance and training time.
    backbone - type of backbone to use
    weights - None or 'imagenet'
    """
    if backbone == "EfNetV2B1":
      base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(include_top=False, input_shape=shape, pooling='max', weights=weights)
    elif backbone == "EfNetV2B2":
      base_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    elif backbone == "ResNet152":
      base_model=tf.keras.applications.resnet_v2.ResNet152V2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    else:
      base_model=tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    for layer in base_model.layers:
        layer.trainable = base_trainable
    dropout_layer = tf.keras.layers.Dropout(
        0.5, name="Dropout1")(base_model.output)
    clf_layer = tf.keras.layers.Dense(
        256, activation="sigmoid", name='Dense1')(dropout_layer)
    dropout_layer2 = tf.keras.layers.Dropout(0.4, name="Dropout2")(clf_layer)
    clf_layer2 = tf.keras.layers.Dense(
        128, activation="sigmoid", name='Dense2')(dropout_layer2)
    dropout_layer3 = tf.keras.layers.Dropout(0.1, name="Dropout3")(clf_layer2)
    clf_layer3 = tf.keras.layers.Dense(
        nc, activation="sigmoid", name='Dense3')(dropout_layer3)
    model = tf.keras.Model(base_model.input, clf_layer3)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return model


def run_model(train_x, train_y, test_x, test_y, val_x, val_y):
    """ A Wrap-up function to run the model. It will be called from the main function in the next cell"""
    file_control(path + 'm.h5')
    image_shape = np.shape(train_x[0])
    num_classes = np.shape(train_y)[1]
    sb = tf.keras.callbacks.ModelCheckpoint(
        path+'m.h5', monitor='val_loss', verbose=1, save_best_only=True)
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0.01, patience=8, restore_best_weights=False)
    model = build_model(nc=num_classes, base_trainable=True, shape=image_shape)
    model.fit(train_x, train_y, batch_size=64, epochs=100, verbose=0,
              callbacks=[es, sb], validation_data=(val_x, val_y))
    model.load_weights(path + 'm.h5')
    return model.predict(test_x)

In [ ]:
def run_experiments(name):
    """
    In this function, we are running the model for each fold of the dataset. The results are stored in the results.pkl file
    name is specyfing the version of the dataset to run experiments on.
    """
    start = time.time()
    print(f'running experiment for {name}')
    results = pd.read_pickle(path + 'results.pkl')
    df = pd.concat(build_df(name), ignore_index=True, axis=0)
    r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
    for fold in results.fold.unique():
        train_x, train_y, test_x, test_y, val_x, val_y = split(df, fold)
        test_link = df[df['fold'] == fold][[
            'slice_file_name', 'fold', 'source']]
        pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y)
        test_link[name] = pd.DataFrame(
            pred, columns=classes).idxmax(axis=1).to_numpy()
        r = pd.concat(
            [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
    results = pd.merge(results, r,  how='left', left_on=[
                       'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
    results.to_pickle(path + "results.pkl")
    end = time.time()
    print(f'done in {str(round(end - start,2))}')

In [ ]:
run_experiments(names[0])

running experiment for dist
cleared old weights

Epoch 1: val_loss improved from inf to 2.16911, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.16911

Epoch 3: val_loss improved from 2.16911 to 1.76334, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.76334 to 1.63157, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.63157 to 1.42430, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.42430

Epoch 7: val_loss improved from 1.42430 to 1.37828, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.37828

Epoch 9: val_loss did not improve from 1.37828

Epoch 10: val_loss improved from 1.37828 to 1.17676, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.17676

Epoch 12: val_loss did not improve from 1.17676

Epoch 13: val_loss did not improve from 1.17676

Epoch 14: val_loss did not improve from 1.17676

Epoch 15: val_l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.22522 to 2.06692, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.06692 to 1.90283, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.90283 to 1.73086, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.73086 to 1.56992, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.56992

Epoch 7: val_loss improved from 1.56992 to 1.55502, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.55502

Epoch 9: val_loss did not improve from 1.55502

Epoch 10: val_loss improved from 1.55502 to 1.52207, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.52207

Epoch 12: val_loss did not improve from 1.52207

Epoch 13: val_loss did not improve from 1.

...


Epoch 2: val_loss improved from 2.05642 to 1.97609, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 1.97609

Epoch 4: val_loss improved from 1.97609 to 1.81736, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.81736

Epoch 6: val_loss improved from 1.81736 to 1.30991, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.30991

Epoch 8: val_loss did not improve from 1.30991

Epoch 9: val_loss did not improve from 1.30991

Epoch 10: val_loss did not improve from 1.30991

Epoch 11: val_loss improved from 1.30991 to 1.29642, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss did not improve from 1.29642

Epoch 13: val_loss did not improve from 1.29642

Epoch 14: val_loss did not improve from 1.29642

Epoch 15: val_loss did not improve from 1.29642

Epoch 16: val_loss improved from 1.29

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.28073 to 1.99018, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.99018 to 1.75830, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.75830 to 1.66716, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.66716 to 1.62078, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.62078 to 1.31155, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.31155

Epoch 8: val_loss improved from 1.31155 to 1.24515, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.24515

Epoch 10: val_loss did not improve from 1.24515

Epoch 11: val_loss did not improve from 1.24515

Epoch 12: val_loss did not improve from 1.24515

Epoch 13: val_loss did not improve from 1.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.25884 to 1.93969, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.93969 to 1.79624, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.79624 to 1.75220, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.75220 to 1.62288, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.62288 to 1.60906, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.60906

Epoch 8: val_loss improved from 1.60906 to 1.54838, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.54838

Epoch 10: val_loss did not improve from 1.54838

Epoch 11: val_loss did not improve from 1.54838

Epoch 12: val_loss did not improve from 1.54838

Epoch 13: val_loss did not improve from 1.

In [ ]:
run_experiments(names[1])

running experiment for mixup
cleared old weights

Epoch 1: val_loss improved from inf to 2.20215, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.20215 to 2.03858, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.03858 to 1.97988, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.97988

Epoch 5: val_loss improved from 1.97988 to 1.86931, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.86931 to 1.82819, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.82819 to 1.72445, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.72445

Epoch 9: val_loss did not improve from 1.72445

Epoch 10: val_loss improved from 1.72445 to 1.68567, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.68567

Epoch 12: val_loss did not improve from 1.68567

Epoch 13: val_loss did not improve from 1.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.23861 to 2.12180, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.12180 to 1.93449, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.93449 to 1.86841, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.86841

Epoch 6: val_loss did not improve from 1.86841

Epoch 7: val_loss improved from 1.86841 to 1.66405, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.66405

Epoch 9: val_loss did not improve from 1.66405

Epoch 10: val_loss improved from 1.66405 to 1.63818, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.63818

Epoch 12: val_loss improved from 1.63818 to 1.51231, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss did not improve from 1.

...


Epoch 2: val_loss improved from 2.20006 to 2.01606, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.01606 to 1.80343, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.80343 to 1.72805, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.72805 to 1.62969, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.62969 to 1.56806, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.56806

Epoch 8: val_loss improved from 1.56806 to 1.42432, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.42432 to 1.35339, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.35339

Epoch 11: val_loss did not improve from 1.35339

Epoch 12: val_loss d

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.19642 to 1.95874, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.95874 to 1.84515, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.84515

Epoch 5: val_loss improved from 1.84515 to 1.78578, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.78578 to 1.58717, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.58717 to 1.53535, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.53535 to 1.53474, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.53474

Epoch 10: val_loss did not improve from 1.53474

Epoch 11: val_loss did not improve from 1.53474

Epoch 12: val_loss did not improve from 1.53474

Epoch 13: val_loss did not improve from 1.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.20733 to 1.89711, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.89711 to 1.73356, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.73356 to 1.69282, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.69282 to 1.49063, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.49063

Epoch 7: val_loss did not improve from 1.49063

Epoch 8: val_loss did not improve from 1.49063

Epoch 9: val_loss improved from 1.49063 to 1.44795, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 1.44795 to 1.43043, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.43043

Epoch 12: val_loss improved from 1.43043 to 1.37094, saving model to /content/gdrive/MyDriv

In [ ]:
run_experiments(names[2])

running experiment for imixup
cleared old weights

Epoch 1: val_loss improved from inf to 2.15437, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.15437 to 1.93523, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 1.93523

Epoch 4: val_loss improved from 1.93523 to 1.84321, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.84321 to 1.73500, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.73500 to 1.67628, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.67628

Epoch 8: val_loss improved from 1.67628 to 1.63205, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.63205

Epoch 10: val_loss improved from 1.63205 to 1.57838, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.57838

Epoch 12: val_loss did not improve from 1.57838

Epoch 13: val_loss did not improve from 1.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.24759 to 2.07578, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.07578 to 1.97710, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.97710 to 1.77737, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.77737 to 1.72347, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.72347 to 1.69625, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.69625

Epoch 8: val_loss improved from 1.69625 to 1.58957, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.58957

Epoch 10: val_loss did not improve from 1.58957

Epoch 11: val_loss did not improve from 1.58957

Epoch 12: val_loss improved from 1.58957 to 1.54245, saving model to /content/gdrive/MyDriv

...


Epoch 2: val_loss improved from 2.23045 to 2.11155, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.11155 to 1.94439, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.94439 to 1.83206, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.83206 to 1.80324, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.80324

Epoch 7: val_loss improved from 1.80324 to 1.57350, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.57350

Epoch 9: val_loss did not improve from 1.57350

Epoch 10: val_loss improved from 1.57350 to 1.52583, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.52583

Epoch 12: val_loss improved from 1.52583 to 1.49005, saving model to /content/gdrive/MyDriv

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.28661 to 2.09876, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 2.09876

Epoch 4: val_loss improved from 2.09876 to 1.96678, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.96678

Epoch 6: val_loss improved from 1.96678 to 1.72942, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.72942

Epoch 8: val_loss improved from 1.72942 to 1.70952, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.70952 to 1.67049, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.67049

Epoch 11: val_loss did not improve from 1.67049

Epoch 12: val_loss did not improve from 1.67049

Epoch 13: val_loss improved from 1.67049 to 1.54761, saving model to /content/gdrive/MyDrive/Colab Notebooks/data

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.25614 to 2.00773, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.00773 to 1.86744, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.86744 to 1.78311, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.78311 to 1.65700, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.65700 to 1.65472, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.65472

Epoch 8: val_loss improved from 1.65472 to 1.60995, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.60995 to 1.56037, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.56037

Epoch 11: val_loss improved from 1.56037 to 1.51221, saving model to 

In [ ]:
run_experiments(names[3])

running experiment for room
cleared old weights

Epoch 1: val_loss improved from inf to 2.21505, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.21505 to 2.09745, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.09745 to 1.87458, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.87458 to 1.61561, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.61561

Epoch 6: val_loss improved from 1.61561 to 1.47208, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.47208 to 1.37756, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.37756 to 1.25842, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.25842 to 1.22959, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.22959

Epoch 11: val_loss improved from 1.22959 to 1.08419, saving model to 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.25594 to 1.81345, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.81345 to 1.55612, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.55612 to 1.29375, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.29375

Epoch 6: val_loss did not improve from 1.29375

Epoch 7: val_loss improved from 1.29375 to 1.14930, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.14930

Epoch 9: val_loss did not improve from 1.14930

Epoch 10: val_loss improved from 1.14930 to 1.14891, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss improved from 1.14891 to 1.08519, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss improved from 1.08519 to 1.00092, saving model to /content/gdrive/MyDriv

...


Epoch 2: val_loss improved from 2.15729 to 1.79263, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.79263 to 1.56422, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.56422 to 1.41790, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.41790 to 1.29474, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.29474

Epoch 7: val_loss improved from 1.29474 to 1.28324, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.28324 to 1.17429, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.17429

Epoch 10: val_loss improved from 1.17429 to 1.09311, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.09311

Epoch 12: val_loss d

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.21021 to 1.94259, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.94259 to 1.78152, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.78152 to 1.70214, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.70214

Epoch 6: val_loss improved from 1.70214 to 1.56147, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.56147 to 1.42759, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.42759 to 1.40964, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.40964 to 1.30487, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.30487

Epoch 11: val_loss did not improve from 1.30487

Epoch 12: val_loss d

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.41603 to 1.87462, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.87462 to 1.80862, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.80862 to 1.76730, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.76730 to 1.43474, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.43474

Epoch 7: val_loss did not improve from 1.43474

Epoch 8: val_loss improved from 1.43474 to 1.14385, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.14385

Epoch 10: val_loss did not improve from 1.14385

Epoch 11: val_loss did not improve from 1.14385

Epoch 12: val_loss did not improve from 1.14385

Epoch 13: val_loss did not improve from 1.14385

Epoch 14: val_loss did not improve from 1.14385

Epoch 15: val_l

In [ ]:
run_experiments(names[4])

running experiment for warp
cleared old weights

Epoch 1: val_loss improved from inf to 2.18456, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.18456 to 2.07176, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.07176 to 1.92903, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.92903 to 1.60620, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.60620

Epoch 6: val_loss improved from 1.60620 to 1.48596, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.48596 to 1.36270, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.36270 to 1.24259, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.24259 to 1.19865, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 1.19865 to 1.15854, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.26975 to 1.99738, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.99738 to 1.77500, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.77500 to 1.67086, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.67086 to 1.47764, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.47764 to 1.25012, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.25012

Epoch 8: val_loss improved from 1.25012 to 1.13066, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.13066 to 1.07396, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.07396

Epoch 11: val_loss improved from 1.07396 to 0.88463, saving model to 

...


Epoch 2: val_loss improved from 2.18399 to 2.06567, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.06567 to 1.91405, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.91405 to 1.60889, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.60889

Epoch 6: val_loss improved from 1.60889 to 1.37455, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.37455

Epoch 8: val_loss did not improve from 1.37455

Epoch 9: val_loss improved from 1.37455 to 1.35242, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 1.35242 to 1.25286, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.25286

Epoch 12: val_loss did not improve from 1.25286

Epoch 13: val_loss improved from 1.25286 t

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.18081 to 1.96054, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.96054 to 1.84340, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.84340 to 1.73530, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.73530 to 1.59507, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.59507 to 1.58210, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.58210 to 1.57000, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.57000 to 1.32940, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss improved from 1.32940 to 1.30270, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.30270

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.21464 to 1.92347, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.92347 to 1.83526, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.83526

Epoch 5: val_loss improved from 1.83526 to 1.72773, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.72773 to 1.46060, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.46060


In [ ]:
run_experiments(names[5])

running experiment for delay
already clear

Epoch 1: val_loss improved from inf to 2.21958, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.21958 to 2.16414, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.16414 to 1.95175, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.95175 to 1.88673, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.88673 to 1.76336, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.76336 to 1.67238, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.67238 to 1.64210, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.64210

Epoch 9: val_loss improved from 1.64210 to 1.63930, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 1.63930 to 1.48984, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.16894 to 1.96972, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.96972 to 1.94753, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.94753 to 1.77598, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.77598 to 1.60813, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.60813 to 1.46212, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.46212 to 1.32973, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.32973

Epoch 9: val_loss did not improve from 1.32973

Epoch 10: val_loss did not improve from 1.32973

Epoch 11: val_loss did not improve from 1.32973

Epoch 12: val_loss did not improve from 1.32973

Epoch 13: val_loss did not improve from 1.

...


Epoch 2: val_loss improved from 2.16978 to 1.92995, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.92995 to 1.66912, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.66912 to 1.48232, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.48232 to 1.22671, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.22671

Epoch 7: val_loss did not improve from 1.22671

Epoch 8: val_loss did not improve from 1.22671

Epoch 9: val_loss improved from 1.22671 to 1.06667, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.06667

Epoch 11: val_loss did not improve from 1.06667

Epoch 12: val_loss did not improve from 1.06667

Epoch 13: val_loss did not improve from 1.06667

Epoch 14: val_loss did not improve from 1.06667

Epoch 15: val_l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.08034 to 1.76845, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.76845 to 1.70932, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.70932 to 1.55250, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.55250 to 1.34501, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.34501 to 1.30937, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.30937

Epoch 8: val_loss did not improve from 1.30937

Epoch 9: val_loss did not improve from 1.30937

Epoch 10: val_loss did not improve from 1.30937

Epoch 11: val_loss did not improve from 1.30937

Epoch 12: val_loss did not improve from 1.30937

Epoch 13: val_loss did not improve from 1.30937

Epoch 14: val_loss did not improve from 1.30937
12/12 [=========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.17523 to 1.82657, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.82657 to 1.74546, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.74546

Epoch 5: val_loss improved from 1.74546 to 1.51403, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.51403 to 1.41636, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.41636 to 1.27152, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.27152

Epoch 9: val_loss did not improve from 1.27152

Epoch 10: val_loss did not improve from 1.27152

Epoch 11: val_loss did not improve from 1.27152

Epoch 12: val_loss did not improve from 1.27152

Epoch 13: val_loss improved from 1.27152 to 1.25911, saving model to /content/gdrive/MyDrive/Colab Notebooks/data

In [ ]:
run_experiments(names[6])

running experiment for spectrum
cleared old weights

Epoch 1: val_loss improved from inf to 2.36457, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.36457 to 2.12278, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.12278 to 2.12269, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 2.12269 to 1.89189, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.89189 to 1.79880, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.79880 to 1.50208, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.50208

Epoch 8: val_loss did not improve from 1.50208

Epoch 9: val_loss did not improve from 1.50208

Epoch 10: val_loss improved from 1.50208 to 1.25143, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.25143

Epoch 12: val_loss did not improve from 1.25143

Epoch 13: val_loss did not improve from 1.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.33721 to 2.11768, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.11768 to 1.92077, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.92077 to 1.83971, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.83971

Epoch 6: val_loss improved from 1.83971 to 1.72256, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.72256 to 1.67360, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.67360

Epoch 9: val_loss improved from 1.67360 to 1.48414, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.48414

Epoch 11: val_loss did not improve from 1.48414

Epoch 12: val_loss did not improve from 1.48414

Epoch 13: val_loss improved from 1.48414 t

...


Epoch 2: val_loss improved from 2.36053 to 2.08780, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.08780 to 1.93845, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.93845 to 1.73134, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.73134

Epoch 6: val_loss improved from 1.73134 to 1.57525, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.57525

Epoch 8: val_loss improved from 1.57525 to 1.56872, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.56872

Epoch 10: val_loss improved from 1.56872 to 1.36509, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.36509

Epoch 12: val_loss did not improve from 1.36509

Epoch 13: val_loss improved from 1.36509 t

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.34577 to 2.15413, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.15413 to 1.92033, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.92033 to 1.89231, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.89231

Epoch 6: val_loss improved from 1.89231 to 1.67008, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.67008

Epoch 8: val_loss improved from 1.67008 to 1.60474, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.60474

Epoch 10: val_loss did not improve from 1.60474

Epoch 11: val_loss did not improve from 1.60474

Epoch 12: val_loss did not improve from 1.60474

Epoch 13: val_loss did not improve from 1.60474

Epoch 14: val_loss improved from 1.60474 to 1.58002, saving mode

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.33153 to 2.20315, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.20315 to 1.97648, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.97648 to 1.86075, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.86075 to 1.80924, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.80924 to 1.65432, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.65432 to 1.40600, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.40600

Epoch 9: val_loss did not improve from 1.40600

Epoch 10: val_loss did not improve from 1.40600

Epoch 11: val_loss did not improve from 1.40600

Epoch 12: val_loss improved from 1.40600 to 1.22364, saving model to /content/gdrive/MyDriv

In [ ]:
pd.read_pickle(path + 'results.pkl').head()

,slice_file_name,fold,class,source,dist,mixup,imixup,room,spectrum,warp,delay,all
0,57320-0-0-39.wav,1,air_conditioner,o,children_playing,air_conditioner,air_conditioner,air_conditioner,children_playing,children_playing,dog_bark,air_conditioner
1,134717-0-0-6.wav,1,air_conditioner,o,street_music,air_conditioner,air_conditioner,air_conditioner,air_conditioner,engine_idling,air_conditioner,engine_idling
2,57320-0-0-31.wav,1,air_conditioner,o,dog_bark,air_conditioner,air_conditioner,dog_bark,dog_bark,dog_bark,dog_bark,air_conditioner
3,134717-0-0-15.wav,1,air_conditioner,o,engine_idling,air_conditioner,air_conditioner,air_conditioner,air_conditioner,air_conditioner,engine_idling,engine_idling
4,177621-0-0-27.wav,1,air_conditioner,o,engine_idling,engine_idling,jackhammer,air_conditioner,engine_idling,jackhammer,jackhammer,air_conditioner


In [ ]:
# In this cell all augmentation methods are combined with the original dataset. A model is then trained on whole dataset.
# Its results are also stored for a comparison.
start = time.time()
name = "all"
results = pd.read_pickle(path + 'results.pkl')
dfs = []
for n in names:
    if n != 'spectrum':
        dfs.append(build_df(n)[1])
    else:
        dfs.append(build_df(n)[0])
df = pd.concat(dfs, ignore_index=True, axis=0)
r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
for fold in results.fold.unique():
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    val = val[val.source == 'o']
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    test = test[test.source == 'o']
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    test_link = test[['slice_file_name', 'fold', 'source']]
    pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y)
    test_link[name] = pd.DataFrame(
        pred, columns=classes).idxmax(axis=1).to_numpy()
    r = pd.concat(
        [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
results = pd.merge(results, r,  how='left', left_on=[
                   'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
results.to_pickle(path + "results.pkl")
end = time.time()
print(f'done in {str(round(end - start,2))}')

cleared old weights

Epoch 1: val_loss improved from inf to 1.67526, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.67526 to 1.07919, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 1.07919

Epoch 4: val_loss did not improve from 1.07919

Epoch 5: val_loss improved from 1.07919 to 1.06077, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.06077

Epoch 7: val_loss improved from 1.06077 to 1.02490, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.02490

Epoch 9: val_loss improved from 1.02490 to 0.95709, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 0.95709

Epoch 11: val_loss did not improve from 0.95709

Epoch 12: val_loss did not improve from 0.95709

Epoch 13: val_loss improved from 0.95709 to 0.79413, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss did not improve from 0.79413

Epoch 15: val_l

<ipython-input-7-df35c4511a8b>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(pred, columns = classes).idxmax(axis=1).to_numpy()


cleared old weights

Epoch 1: val_loss improved from inf to 1.96920, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...


Epoch 2: val_loss improved from 1.93548 to 1.18440, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.18440 to 1.16884, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.16884

Epoch 5: val_loss improved from 1.16884 to 0.82971, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 0.82971 to 0.82881, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 0.82881 to 0.82795, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 0.82795

Epoch 9: val_loss did not improve from 0.82795

Epoch 10: val_loss did not improve from 0.82795

Epoch 11: val_loss did not improve from 0.82795

Epoch 12: val_loss did not improve from 0.82795

Epoch 13: val_loss did not improve from 0.82795
39/39 [==============================] - 3s 20ms/step


<ipython-input-7-df35c4511a8b>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(pred, columns = classes).idxmax(axis=1).to_numpy()


cleared old weights

Epoch 1: val_loss improved from inf to 1.70103, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.70103 to 1.44117, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.44117 to 1.23241, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.23241 to 1.09410, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.09410

Epoch 6: val_loss improved from 1.09410 to 0.99827, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 0.99827

Epoch 8: val_loss did not improve from 0.99827

Epoch 9: val_loss improved from 0.99827 to 0.90617, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 0.90617

Epoch 11: val_loss did not improve from 0.90617

Epoch 12: val_loss did not improve from 0.90617

Epoch 13: val_loss did not improve from 0.90617

Epoch 14: val_loss did not improve from 0.90617

Epoch 15: val_l

In [ ]:
# In this cell all augmentation methods are combined with the original dataset. A model is then trained on whole dataset.
# Its results are also stored for a comparison.
start = time.time()
name = "image_only"
results = pd.read_pickle(path + 'results.pkl')
dfs = []
ni = ['imixup','warp','spectrum']
for n in ni:
    if n != 'spectrum':
        dfs.append(build_df(n)[1])
    else:
        dfs.append(build_df(n)[0])
df = pd.concat(dfs, ignore_index=True, axis=0)
r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
for fold in results.fold.unique():
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    val = val[val.source == 'o']
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    test = test[test.source == 'o']
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    test_link = test[['slice_file_name', 'fold', 'source']]
    pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y)
    test_link[name] = pd.DataFrame(
        pred, columns=classes).idxmax(axis=1).to_numpy()
    r = pd.concat(
        [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
results = pd.merge(results, r,  how='left', left_on=[
                   'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
results.to_pickle(path + "results.pkl")
end = time.time()
print(f'done in {str(round(end - start,2))}')

cleared old weights

Epoch 1: val_loss improved from inf to 1.95555, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.95555 to 1.62350, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.62350 to 1.58152, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.58152 to 1.20364, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.20364

Epoch 6: val_loss improved from 1.20364 to 1.03112, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.03112

Epoch 8: val_loss improved from 1.03112 to 0.96001, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 0.96001

Epoch 10: val_loss did not improve from 0.96001

Epoch 11: val_loss improved from 0.96001 to 0.95693, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss improved from 0.95693 to 0.94818, saving model to /content/gdrive/MyDriv

<ipython-input-8-2306e5aa7842>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 1.98362, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-8-2306e5aa7842>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 1.93946, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.93946 to 1.80951, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.80951 to 1.51424, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.51424

Epoch 5: val_loss improved from 1.51424 to 1.17169, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.17169 to 1.13415, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.13415 to 0.91583, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 0.91583

Epoch 9: val_loss did not improve from 0.91583

Epoch 10: val_loss did not improve from 0.91583

Epoch 11: val_loss improved from 0.91583 to 0.84365, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss did not improve from 0.84365

Epoch 13: val_loss did not improve from 0.

<ipython-input-8-2306e5aa7842>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


In [ ]:
# In this cell all augmentation methods are combined with the original dataset. A model is then trained on whole dataset.
# Its results are also stored for a comparison.
start = time.time()
name = "audio_only"
results = pd.read_pickle(path + 'results.pkl')
dfs = []
na = ['dist', 'mixup', 'room','delay', 'spectrum']
for n in na:
    if n != 'spectrum':
        dfs.append(build_df(n)[1])
    else:
        dfs.append(build_df(n)[0])
df = pd.concat(dfs, ignore_index=True, axis=0)
r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
for fold in results.fold.unique():
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    val = val[val.source == 'o']
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    test = test[test.source == 'o']
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    test_link = test[['slice_file_name', 'fold', 'source']]
    pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y)
    test_link[name] = pd.DataFrame(
        pred, columns=classes).idxmax(axis=1).to_numpy()
    r = pd.concat(
        [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
results = pd.merge(results, r,  how='left', left_on=[
                   'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
results.to_pickle(path + "results.pkl")
end = time.time()
print(f'done in {str(round(end - start,2))}')

cleared old weights

Epoch 1: val_loss improved from inf to 1.82068, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.82068 to 1.34992, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.34992 to 1.22368, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 1.22368

Epoch 5: val_loss improved from 1.22368 to 1.22259, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 1.22259 to 1.12327, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.12327

Epoch 8: val_loss did not improve from 1.12327

Epoch 9: val_loss did not improve from 1.12327

Epoch 10: val_loss did not improve from 1.12327

Epoch 11: val_loss did not improve from 1.12327

Epoch 12: val_loss did not improve from 1.12327

Epoch 13: val_loss did not improve from 1.12327

Epoch 14: val_loss did not improve from 1.12327
29/29 [==============================] - 2s 17ms/step


<ipython-input-9-953fd46bf520>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 1.83200, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-9-953fd46bf520>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 1.63879, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.63879 to 1.19828, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 1.19828

Epoch 4: val_loss improved from 1.19828 to 1.07564, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.07564 to 1.01873, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.01873

Epoch 7: val_loss improved from 1.01873 to 0.92680, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 0.92680 to 0.85569, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 0.85569

Epoch 10: val_loss did not improve from 0.85569

Epoch 11: val_loss did not improve from 0.85569

Epoch 12: val_loss did not improve from 0.85569

Epoch 13: val_loss did not improve from 0.85569

Epoch 14: val_loss did not improve from 0.85569

Epoch 15: val_l

<ipython-input-9-953fd46bf520>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


In [ ]:
# In this cell all augmentation methods are combined with the original dataset. A model is then trained on whole dataset.
# Its results are also stored for a comparison.
start = time.time()
name = "audio_only_nd"
results = pd.read_pickle(path + 'results.pkl')
dfs = []
na = ['dist', 'mixup', 'room', 'spectrum']
for n in na:
    if n != 'spectrum':
        dfs.append(build_df(n)[1])
    else:
        dfs.append(build_df(n)[0])
df = pd.concat(dfs, ignore_index=True, axis=0)
r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
for fold in results.fold.unique():
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    val = val[val.source == 'o']
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    test = test[test.source == 'o']
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    test_link = test[['slice_file_name', 'fold', 'source']]
    pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y)
    test_link[name] = pd.DataFrame(
        pred, columns=classes).idxmax(axis=1).to_numpy()
    r = pd.concat(
        [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
results = pd.merge(results, r,  how='left', left_on=[
                   'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
results.to_pickle(path + "results.pkl")
end = time.time()
print(f'done in {str(round(end - start,2))}')

cleared old weights

Epoch 1: val_loss improved from inf to 1.86146, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.86146 to 1.73077, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.73077 to 1.33753, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.33753 to 1.20871, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 1.20871

Epoch 6: val_loss improved from 1.20871 to 1.09460, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.09460

Epoch 8: val_loss improved from 1.09460 to 0.96043, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 0.96043

Epoch 10: val_loss improved from 0.96043 to 0.92009, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss improved from 0.92009 to 0.78215, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss d

<ipython-input-6-e0bbe73fdc43>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 1.89511, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-6-e0bbe73fdc43>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.04009, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.04009 to 1.79528, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 1.79528 to 1.27325, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 1.27325 to 1.24727, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 1.24727 to 1.14826, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 1.14826

Epoch 7: val_loss did not improve from 1.14826

Epoch 8: val_loss did not improve from 1.14826

Epoch 9: val_loss did not improve from 1.14826

Epoch 10: val_loss did not improve from 1.14826

Epoch 11: val_loss did not improve from 1.14826

Epoch 12: val_loss did not improve from 1.14826

Epoch 13: val_loss did not improve from 1.14826
23/23 [==============================] - 2s 15ms/step
done in 3130.27


<ipython-input-6-e0bbe73fdc43>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(
